In [1]:
!pip install datasets

In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [3]:
from datasets import Dataset, DatasetDict
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [5]:
!gdown 1XGLAnvZt9oEYCscQcqHjbva0k5S605BZ
!gdown 1EEBfIUZD0q0wRXo8pPFnIGzlzaexI6si

Downloading...
From: https://drive.google.com/uc?id=1XGLAnvZt9oEYCscQcqHjbva0k5S605BZ
To: /content/test_data.csv
100% 176k/176k [00:00<00:00, 102MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EEBfIUZD0q0wRXo8pPFnIGzlzaexI6si
To: /content/train_data.csv
100% 692k/692k [00:00<00:00, 138MB/s]


In [6]:
train_df = pd.read_csv("train_data.csv")
train_df.head()

,clean_text,target
0,reluctant start one really ashamed ask help on...,1
1,severe addict overdose young addictionalcoholi...,1
2,feel sad feel sad pray get help desperately ne...,1
3,sleep car month month back foot stay parking s...,1
4,speed try catch continue yell face girl turn a...,0


In [7]:
test_df = pd.read_csv("test_data.csv")
test_df.head()

,clean_text,target
0,abuse end year ago life good get marry someone...,0
1,hi health issue miss lot school eventually sta...,1
2,know want curse hit angry come sister home wan...,1
3,never feel uncomfortable scared night put hand...,1
4,deal ptsd rape recently find someone think fri...,1


In [8]:
train_df = train_df.rename(columns={"clean_text": "text", "target":"label"})
train_df.head()

,text,label
0,reluctant start one really ashamed ask help on...,1
1,severe addict overdose young addictionalcoholi...,1
2,feel sad feel sad pray get help desperately ne...,1
3,sleep car month month back foot stay parking s...,1
4,speed try catch continue yell face girl turn a...,0


In [9]:
test_df = test_df.rename(columns={"clean_text": "text", "target":"label"})
test_df.head()

,text,label
0,abuse end year ago life good get marry someone...,0
1,hi health issue miss lot school eventually sta...,1
2,know want curse hit angry come sister home wan...,1
3,never feel uncomfortable scared night put hand...,1
4,deal ptsd rape recently find someone think fri...,1


In [10]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [11]:
train_ds[10]

{'text': 'silent sadness feel apparent reason yet somehow feel weak much pussy anything even think deserve well feel like deserve whatever painful feeling go feel like rather suffer long painful life rather end life feel like deserve whatever agony go anxiety panic attack come go whenever please sometimes feel like somehow hit invisible truck body frozen somebody turn popsicle choice shake inside',
 'label': 1}

In [16]:
def order(inp):
  '''
  This function will group all the inputs of BERT
  into a single dictionary and then output it with
  labels.
  '''
  data = list(inp.values())
  return {
      'input_ids': data[1],
      'attention_mask': data[2],
      'token_type_ids': data[3]
  }, data[0]

def config_datasets(train_ds, test_ds, tokenizer):
  def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

  train_ds = train_ds.map(tokenize, batched=True, batch_size=None)
  test_ds = test_ds.map(tokenize, batched=True, batch_size=None)

  train_ds.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
  test_ds.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

  BATCH_SIZE = 64



  # converting train split of `emotions_encoded` to tensorflow format
  train_dataset = tf.data.Dataset.from_tensor_slices(train_ds[:])
  # set batch_size and shuffle
  train_dataset = train_dataset.batch(BATCH_SIZE)
  # map the `order` function
  train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

  # ... doing the same for test set ...
  test_dataset = tf.data.Dataset.from_tensor_slices(test_ds[:])
  test_dataset = test_dataset.batch(BATCH_SIZE)
  test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)
  return train_dataset, test_dataset

In [25]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [19]:
class BERTForClassification(tf.keras.Model):

  def __init__(self, bert_model, num_classes):
    super().__init__()
    self.bert = bert_model
    # self.dropout = tf.keras.layers.Dropout(0.2)  # Dropout layer with 0.2 rate
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.bert(inputs)[1]  # Access pooled output from BERT
    # x = self.dropout(x)  # Apply dropout to the pooled output
    # x = self.h1(x)
    return self.fc(x)

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [20]:
classifier = BERTForClassification(model, num_classes=2)
train_dataset, test_dataset = config_datasets(train_ds, test_ds, tokenizer)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Map:   0%|          | 0/2842 [00:00<?, ? examples/s]

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

In [21]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = classifier.fit(
    train_dataset,
    epochs=10,
    validation_data=(test_dataset),
    callbacks=[early_stop]
)

Epoch 1/10
45/45 [==============================] - 155s 2s/step - loss: 0.6905 - accuracy: 0.5415 - val_loss: 0.6194 - val_accuracy: 0.6779
Epoch 2/10
45/45 [==============================] - 97s 2s/step - loss: 0.6257 - accuracy: 0.6545 - val_loss: 0.5987 - val_accuracy: 0.6624
Epoch 3/10
45/45 [==============================] - 97s 2s/step - loss: 0.5161 - accuracy: 0.7569 - val_loss: 0.5264 - val_accuracy: 0.7595
Epoch 4/10
45/45 [==============================] - 97s 2s/step - loss: 0.4254 - accuracy: 0.8177 - val_loss: 0.5240 - val_accuracy: 0.7412
Epoch 5/10
45/45 [==============================] - 96s 2s/step - loss: 0.3007 - accuracy: 0.8888 - val_loss: 0.5328 - val_accuracy: 0.7525
Epoch 6/10
45/45 [==============================] - 97s 2s/step - loss: 0.2506 - accuracy: 0.9131 - val_loss: 0.8137 - val_accuracy: 0.6681
Epoch 7/10
45/45 [==============================] - 97s 2s/step - loss: 0.2402 - accuracy: 0.9008 - val_loss: 0.7392 - val_accuracy: 0.6962
Epoch 8/10
45/45 [=

In [26]:
y_pred = classifier.predict(test_dataset)
y_pred = (y_pred > 0.5).astype(int)


12/12 [==============================] - 6s 515ms/step


In [29]:
get_report(test_ds['label'], y_pred)

{'Accuracy': 0.7412095639943741,
 'Precision': 0.7146666666666667,
 'Recall': 0.7768115942028986,
 'F1-Score': 0.7444444444444444}